In [11]:
import pandas as pd
import fnmatch
import os
import csv
import codecs
import boto3
import io
from io import StringIO

In [12]:
# Load and Cleansing of Training Data

In [13]:
# Load from S3
!ln -s /Users/matthiashugli/Virtualenvs/youth-base/youth-base/config.py config.py
from config import s3


ln: config.py: File exists


In [86]:
# Get all CSV Files in the subfolder rawdata/
bucket_name = s3.Bucket('training-minutes')

bucket_list = []
for file in bucket_name.objects.filter(Prefix = 'rawdata/'):
    file_key = file.key
    if file_key.find('.csv') != -1:
        bucket_list.append(file.key)
length_bucket_list = print(len(bucket_list))
print(bucket_name)

2
s3.Bucket(name='training-minutes')


In [119]:
df = pd.DataFrame()
for file_name in bucket_list:
    obj = s3.Object(bucket_name.name, file_name)
    data = obj.get()['Body'].read()
    file = pd.read_csv(io.BytesIO(data), header=1, delimiter=',', low_memory=False)

    df = df.append(file)

## Save to csv file as staging file
df.to_csv('trainings.csv', encoding='utf-8')
csv_buffer = StringIO()
s3.Object(bucket_name.name, 'staging_trainings.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'AQHR2FJQENX6MD3Z',
  'HostId': 'xieVhMC3CVXEhJxLSpWf0yfxviSNIFoKSSLlErLN8NNdJZiEZwB/NS3mJN/Tpj1g3xcsj1+voz8=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'xieVhMC3CVXEhJxLSpWf0yfxviSNIFoKSSLlErLN8NNdJZiEZwB/NS3mJN/Tpj1g3xcsj1+voz8=',
   'x-amz-request-id': 'AQHR2FJQENX6MD3Z',
   'date': 'Thu, 25 Mar 2021 06:20:10 GMT',
   'etag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"'}

In [117]:
df.columns = df.iloc[0]

df = df[1:]
df = df.melt(df.columns[0:3])

print(df.head(3))
df.to_csv('test.csv', encoding='utf-8')

   FE-14 BSC YOUNG BOYS       Jan Uebersax  Tor    0 value
0  FE-14 BSC YOUNG BOYS   Elia Pietropaolo  Tor  NaN   NaN
1    U17 BSC Young Boys  Mario Baumgartner  Tor  NaN   NaN
2    U17 BSC Young Boys  Elio Castro Baldo  Tor  NaN   NaN


In [ ]:
    file = file.iloc[2:]
    file = file.rename(columns=lambda c: c.replace('.1', '.number_trainings') if fnmatch.fnmatch(c, '*.1') else c + ".duration")
    file = file.melt(file.columns[0:3])
    file.insert(0, 'filename', file_name)

In [83]:
# Load from S3
!ln -s /Users/matthiashugli/Virtualenvs/youth-base/youth-base/config.py config.py
from config import s3

bucket_name = 'training-minutes'

df = pd.DataFrame()
for obj in s3.Bucket(bucket_name).objects.all():
    ## List all files and convert them to Dataframe
    if obj.key[-5:] == '.csv':
        data = s3.Bucket(bucket_name).Object(obj.key).get()
        file = pd.read_csv(data['body'])
        filename = obj.key

        ## Cleansing of File
        ### Remove unnecessary rows, label duration and count measure columns correctly and transpose into long
        file = file.iloc[2:]
        file = file.rename(columns=lambda c: c.replace('.1', '.number_trainings') if fnmatch.fnmatch(c, '*.1') else c + ".duration")
        file = file.melt(file.columns[0:3])

        ### Properly Name columns and categories
        file[['skills', 'entity']] = file.variable.apply(lambda x: pd.Series(str(x).split('.')))
        file = file.drop(columns=['Category Name.duration', 'variable'])
        file = file.rename(columns={'Unnamed: 0.duration': 'team', 'Unnamed: 1.duration': 'keeper'})

        ### Pivot file for all categories and create two columns for duration in minutes and number of trainings
        file = file.pivot(index=['team', 'keeper', 'skills'], columns='entity', values='value').reset_index()

        ### Clean catgegory and date columns
        file[['team', 'club']] = file.team.str.split(' BSC', 1, expand=True)
        file[['team', 'club']] = file.team.str.split(' YB', 1, expand=True)
        file['filename'] = filename
        file[['file', 'type']] = file.filename.apply(lambda x: pd.Series(str(x).split('.')))
        file['yearmonth'] = file.file.str[-6:]
        file['date'] = pd.to_datetime(file.yearmonth, format='%Y%m', errors='coerce').dropna()
        file['year'] = pd.DatetimeIndex(file['date']).year
        file['month'] = file.date.dt.month_name()

        ### Drop all helping columns and values with NaN
        file = file.drop(columns=['filename', 'file', 'type', 'club'])
        file = file.dropna(subset=['number_trainings', 'duration'])

        ### append cleansed file into one Dataframe
        df = df.append(file)

## Save to csv file
df.to_csv('trainings.csv', encoding='utf-8')
csv_buffer = StringIO()
s3.Object(bucket_name, 'trainings.csv').put(Body=csv_buffer.getvalue())
   

ln: config.py: File exists


{'ResponseMetadata': {'RequestId': '5227Y3FZMVKMW43X',
  'HostId': 'VfRf1sZpFyElgjpeLratLWn8M44ObjnjaLFUd+rO8BRnMnSnf7VtvKc26GKzvBK4uK9azuXJTo0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'VfRf1sZpFyElgjpeLratLWn8M44ObjnjaLFUd+rO8BRnMnSnf7VtvKc26GKzvBK4uK9azuXJTo0=',
   'x-amz-request-id': '5227Y3FZMVKMW43X',
   'date': 'Thu, 25 Mar 2021 06:07:21 GMT',
   'etag': '"d41d8cd98f00b204e9800998ecf8427e"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"d41d8cd98f00b204e9800998ecf8427e"'}

In [4]:
# Outdated!!! Read from local files, however files are stored on S3 from March 2021

path = '/Users/matthiashugli/Virtualenvs/youth-base/youth-base/training-analysis/data'

data = [os.path.join(dirpath, f) for dirpath, dirnames, files in os.walk(path) for f in fnmatch.filter(files, '*.xlsx')]

df = pd.DataFrame()
for filename in data:
    file = pd.read_excel(filename, sheet_name='s1_core_trm_player_unit_skills', index_col=None, header=1)


In [5]:
# ETL Job in AWS glue
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

## @params: [TempDir, JOB_NAME]
args = getResolvedOptions(sys.argv, ['JOB_NAME'])

glueContext = glueContext(SparkContext.getOrCreate())
glueJob = Job(glueContext)

glueJob.init(args['JOB_NAME'], args)

## ETL
!ln -s /Users/matthiashugli/Virtualenvs/youth-base/youth-base/config.py config.py
from config import s3

bucket_name = 'training-minutes'

df = pd.DataFrame()
for obj in s3.Bucket(bucket_name).objects.all():
    ## List all files and convert them to Dataframe
    data = s3.Bucket(bucket_name).Object(obj.key).get()
    file = pd.read_excel(io.BytesIO(data['Body'].read()), sheet_name='s1_core_trm_player_unit_skills', index_col=None, header=1)
    filename = obj.key

    ## Cleansing of File
    ### Remove unnecessary rows, label duration and count measure columns correctly and transpose into long
    file = file.iloc[2:]
    file = file.rename(columns=lambda c: c.replace('.1', '.number_trainings') if fnmatch.fnmatch(c, '*.1') else c + ".duration")
    file = file.melt(file.columns[0:3])

    ### Properly Name columns and categories
    file[['skills', 'entity']] = file.variable.apply(lambda x: pd.Series(str(x).split('.')))
    file = file.drop(columns=['Category Name.duration', 'variable'])
    file = file.rename(columns={'Unnamed: 0.duration': 'team', 'Unnamed: 1.duration': 'keeper'})

    ### Pivot file for all categories and create two columns for duration in minutes and number of trainings
    file = file.pivot(index=['team', 'keeper', 'skills'], columns='entity', values='value').reset_index()

    ### Clean catgegory and date columns
    file[['team', 'club']] = file.team.str.split(' BSC', 1, expand=True)
    file[['team', 'club']] = file.team.str.split(' YB', 1, expand=True)
    file['filename'] = filename
    file[['file', 'type']] = file.filename.apply(lambda x: pd.Series(str(x).split('.')))
    file['yearmonth'] = file.file.str[-6:]
    file['date'] = pd.to_datetime(file.yearmonth, format='%Y%m', errors='coerce').dropna()
    file['year'] = pd.DatetimeIndex(file['date']).year
    file['month'] = file.date.dt.month_name()

    ### Drop all helping columns and values with NaN
    file = file.drop(columns=['filename', 'file', 'type', 'club'])
    file = file.dropna(subset=['number_trainings', 'duration'])

    ### append cleansed file into one Dataframe
    df = df.append(file)

print(df.head(3))



glueJob.commit()


ModuleNotFoundError: No module named 'awsglue'